In [1]:
import numpy
import numpy.random as rnd
import json
import pandas as pd
import itertools

In [2]:
%run logic.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [ ]:
class Participant:
    poll = None
    matrices = None
    epsilons = None
    budget = 0.
    cost = 0
    
    real_answer = {}
    
    def __init__(self, budget, poll):
        self.budget = budget # Initial privacy budget
        self.poll = poll
        
        self.matrices = pollToMatrix(poll)
        self.epsilons = pollToEpsilon(poll)
        
        self.cost = sum([self.epsilons[x] for x in self.epsilons.keys()])
        
    def createAnswer(self):
        matrices = pollToMatrix(self.poll)
        answer_space = pollToPaths(self.poll)
        weights = pollToPathWeights(self.poll)
        answer = None
        for question in matrices:
            space = answer_space[question]
            temp = weights[question]
            p = []

            for elem in space:
                path = None

                # Find concatenated version
                key = pathToKey(elem)
                p.append(temp[key])

            self.real_answer[question] = rnd.choice(space, p=p)
        
    def randomizedResponse(self):
        
        # Only reply if we can afford it
        responses = {}
        answer_space = pollToPaths(self.poll)
        
        if (self.cost <= self.budget):
            self.budget = self.budget - self.cost
            
            for question in self.matrices.keys():
                transitions = self.matrices[question]
                match = pathToKey(self.real_answer[question])

                row = {}
                for (_input, _output) in transitions.keys():
                    if match == _input:
                        row[_output] = transitions[(_input, _output)]

                p=[]        
                for ans in answer_space[question]:
                    # Match tuples, should be zip:able
                    # zip(answer_space[0], p)
                    p.append(row[pathToKey(ans)])
                response = rnd.choice(answer_space[question], p=p)
                responses[question] = response
                
            return responses
        
        # Budgets are public, so we can refuse to answer without leaking information
        return None
    
    def getTrueAnswers(self):
        return self.real_answer
    
    

participant = Participant(120, example_json_poll)
participant.createAnswer()
print participant.real_answer

participant.randomizedResponse()